In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Fuck warnings
import warnings
warnings.filterwarnings('ignore')

## Building a master dataset

In [10]:
master_df = pd.DataFrame()

## Basketball Reference

In [84]:
# Load data for each of the https://www.basketball-reference.com/ sites
br_per100_url = "https://www.basketball-reference.com/leagues/NBA_2018_per_poss.html"
br_per100_data = pd.read_html(br_per100_url)[0]
cols_list = br_per100_data.columns.tolist()
cols_100 = [elm for elm in cols_list[8:]]
cols_100.append( cols_list[1] )

br_adv_url  = "https://www.basketball-reference.com/leagues/NBA_2018_advanced.html"
br_adv_data = pd.read_html(br_adv_url)[0]
cols_adv = br_adv_data.columns.tolist()[1:3] + br_adv_data.columns.tolist()[4:]


br_tot_url = "https://www.basketball-reference.com/leagues/NBA_2018_totals.html"
br_tot_data = pd.read_html(br_tot_url)[0]

tmp = br_tot_data.columns.tolist()
tmp = tmp[1:]
not_keep = tmp[1:7]
final = list()
for elm in tmp:
    if elm not in not_keep:
        final.append( elm )
        
final_2 = list()
for elm in final:
    if ('%' not in elm) or (elm == "eFG%"):
        final_2.append(elm)
        
cols_tot = final_2

In [4]:
import requests

# Load data for each of the http://stats.nba.com/leaders/
nba_stats_total_url = "http://stats.nba.com/leaders/?Season=2017-18&SeasonType=Regular%20Season&PerMode=Totals"
address = nba_stats_total_url

response = requests.get(address, headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:46.0) Gecko/20100101 Firefox/46.0',
})
response.raise_for_status()         # Raise exception if invalid   response. 
#data = response.json()              # JSON decoding. 

## NBA Stats Scrape

In [5]:
nba_stats_total_url = "http://stats.nba.com/leaders/?Season=2017-18&SeasonType=Regular%20Season&PerMode=Totals"

In [6]:
%matplotlib inline
from selenium import webdriver
from pandas import *
import pandas
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import *

In [7]:
## Set up chrome driver 
path_to_chromedriver = "./web_scraping/chromedriver"
browser = webdriver.Chrome(executable_path=path_to_chromedriver)

In [8]:
## Open browser connection
url = 'https://stats.nba.com/leaders'
browser.get(url)

In [9]:
## Change the parameters to whatever you like

## This is for the correct seasson year
browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/div[1]/div[2]/div/div/label/select/option[2]').click()
## This is for "all" players option 
browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[3]/div/div/select/option[1]').click()

## Follow up, just need to prompt user to enter the season you want to look at and such.

In [10]:
# Get table from HTML
table = browser.find_element_by_class_name('nba-stat-table__overflow')

In [11]:
player_ids = []
player_names = []
player_stats = []

for line_id, lines in enumerate(table.text.split('\n')):
    if line_id == 0:
        column_names = lines.split(' ')[1:]
    else:
        if line_id % 3 == 1:
            player_ids.append(lines)
        if line_id % 3 == 2:
            player_names.append(lines)
        if line_id % 3 == 0:
            player_stats.append( [float(i) for i in lines.split(' ')] )

In [12]:
db = pandas.DataFrame({'player': player_names,
                       'gp': [i[0] for i in player_stats],
                       'min': [i[1] for i in player_stats],
                       'pts': [i[2] for i in player_stats],
                       'fgm': [i[3] for i in player_stats], 
                       'fga': [i[4] for i in player_stats],
                       'fg%': [i[5] for i in player_stats],
                       '3pm': [i[6] for i in player_stats],
                       '3pa': [i[7] for i in player_stats],
                       '3p%': [i[8] for i in player_stats],
                       'ftm': [i[9] for i in player_stats],
                       'fta': [i[10] for i in player_stats],
                       'ft%': [i[11] for i in player_stats],
                       'oreb': [i[12] for i in player_stats],
                       'dreb': [i[13] for i in player_stats],
                       'reb': [i[14] for i in player_stats],
                       'ast': [i[15] for i in player_stats],
                       'stl': [i[16] for i in player_stats],
                       'blk': [i[17] for i in player_stats],
                       'tov': [i[18] for i in player_stats],
                       'eff': [i[19] for i in player_stats]
                       }
                     )

In [13]:
db = db[['player', 
         'gp', 
         'min', 
         'pts', 
         'fgm', 
         'fga', 
         'fg%', 
         '3pm', 
         '3pa', 
         '3p%', 
         'ftm',
         'fta', 
         'ft%', 
         'oreb', 
         'dreb',
         'reb',
         'ast',
         'stl',
         'blk',
         'tov',
         'eff']
      ]

In [14]:
db.head()

,player,gp,min,pts,fgm,fga,fg%,3pm,3pa,3p%,...,fta,ft%,oreb,dreb,reb,ast,stl,blk,tov,eff
0,James Harden,72.0,35.4,30.4,9.0,20.1,44.9,3.7,10.0,36.7,...,10.1,85.8,0.6,4.8,5.4,8.8,1.8,0.7,4.4,30.1
1,Anthony Davis,75.0,36.4,28.1,10.4,19.5,53.4,0.7,2.2,34.0,...,8.0,82.8,2.5,8.6,11.1,2.3,1.5,2.6,2.2,33.0
2,LeBron James,82.0,36.9,27.5,10.5,19.3,54.2,1.8,5.0,36.7,...,6.5,73.1,1.2,7.5,8.6,9.1,1.4,0.9,4.2,32.7
3,Damian Lillard,73.0,36.6,26.9,8.5,19.4,43.9,3.1,8.6,36.1,...,7.4,91.6,0.8,3.6,4.5,6.6,1.1,0.4,2.8,25.0
4,Giannis Antetokounmpo,75.0,36.7,26.9,9.9,18.7,52.9,0.6,1.9,30.7,...,8.5,76.0,2.1,8.0,10.0,4.8,1.5,1.4,3.0,30.7


Next (Data Storage): 
Each player in a row: features including defense
Team stats + have an accurate list of who's on what team
NBA Schedule so you know what to run it on

Next next (Math/Competition Structure):
fan duel formula, what's being predicted, etc.

customize minutes
run individual games instead of all at once
    Early slate: 700pm games, etc. 
    SC Questionable for 10pm game, don't put him in lineups late slight bc who knows if he'll play
    
## Scrape the box scores

In [16]:
browser2 = webdriver.Chrome(executable_path=path_to_chromedriver)

Categorizing players then try to find ts for their minutes

In [19]:
rg_url_1 = "https://rotogrinders.com/grids/nba-defense-vs-position-cheat-sheet-1493632?site=fanduel"
rg_url_2 = "https://rotogrinders.com/game-stats/nba/consistency"

In [21]:
browser2.get(rg_url_2)

In [37]:
from bs4 import BeautifulSoup
import urllib3

http = urllib3.PoolManager()

response1 = http.request('GET', rg_url_1)
rg1_soup = BeautifulSoup(response1.data, 'html.parser')

response2 = http.request('GET', rg_url_2)
rg2_soup = BeautifulSoup(response2.data, 'html.parser')

In [84]:
x = rg2_soup.find("div", attrs={"class": "rgt-col"})

In [60]:
# We do need to use chromdriver to click "all" on this website
# And also need to click "Last 2 seasons"

## ESPN Data

In [88]:
espn_url = "http://www.espn.com/nba/statistics/rpm/_/sort/RPM"
espn_data = pd.read_html(espn_url)[0]
espn_data.columns = espn_data.iloc[0,:].tolist()
espn_data = espn_data.iloc[1:,:]
espn_data.index = pd.RangeIndex(0, len(espn_data))
needed_espn_data = espn_data[ ["NAME", "ORPM", "DRPM", "RPM", "WINS"] ]